In [8]:
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path("..").resolve()
DATA_ROOT = PROJECT_ROOT / "data_csv"
CURIEUS_ROOT = DATA_ROOT / "curieus"

In [9]:
REFERENCE_IDS = ["001", "045", "183"]

REFERENCE_PATHS = {
    rid: CURIEUS_ROOT / rid
    for rid in REFERENCE_IDS
}

REFERENCE_PATHS

def load_curieus_outputs(base_path: Path):
    results = {}

    # developability 계열
    for f in base_path.glob("*develop*.csv"):
        results["developability"] = pd.read_csv(f)

    # viscosity 계열
    for f in base_path.glob("*viscosity*.csv"):
        results["viscosity"] = pd.read_csv(f)

    return results

REFERENCE_RESULTS = {
    rid: load_curieus_outputs(path)
    for rid, path in REFERENCE_PATHS.items()
}

REFERENCE_RESULTS.keys()

dict_keys(['001', '045', '183'])

In [10]:
summary_check = []

for rid, outputs in REFERENCE_RESULTS.items():
    summary_check.append({
        "antibody_id": rid,
        "has_developability": "developability" in outputs,
        "has_viscosity": "viscosity" in outputs,
    })

pd.DataFrame(summary_check)

,antibody_id,has_developability,has_viscosity
0,001,False,False
1,045,False,False
2,183,False,False


In [11]:
def summarize_developability(df: pd.DataFrame):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    return df[numeric_cols].describe().loc[["mean", "min", "max"]]

developability_summary = {}

for rid, outputs in REFERENCE_RESULTS.items():
    if "developability" in outputs:
        developability_summary[rid] = summarize_developability(
            outputs["developability"]
        )

developability_summary

{}

In [12]:
reference_risk_profile = []

for rid in REFERENCE_IDS:
    outputs = REFERENCE_RESULTS.get(rid, {})
    
    risk_notes = []

    if "viscosity" in outputs:
        risk_notes.append("Viscosity signal present")

    if "developability" in outputs:
        risk_notes.append("Developability constraints observed")

    reference_risk_profile.append({
        "antibody_id": rid,
        "risk_context": "; ".join(risk_notes) if risk_notes else "No strong signal detected",
    })

reference_risk_df = pd.DataFrame(reference_risk_profile)
reference_risk_df

,antibody_id,risk_context
0,001,No strong signal detected
1,045,No strong signal detected
2,183,No strong signal detected


In [13]:
CORE7_POLICY_REFERENCE = {
    "VETO_SOMS": 1.5,
    "FREEZE_SOMS": 2.5,
    "VETO_CONFLICT_STREAK": 1,
    "FREEZE_VETO_STREAK": 2,
}

policy_context = pd.DataFrame([
    {
        "policy_threshold": k,
        "value": v,
        "interpretation": "Conservative governance threshold informed by prior Curieus instability patterns"
    }
    for k, v in CORE7_POLICY_REFERENCE.items()
])

policy_context

,policy_threshold,value,interpretation
0,VETO_SOMS,1.5,Conservative governance threshold informed by ...
1,FREEZE_SOMS,2.5,Conservative governance threshold informed by ...
2,VETO_CONFLICT_STREAK,1.0,Conservative governance threshold informed by ...
3,FREEZE_VETO_STREAK,2.0,Conservative governance threshold informed by ...


In [14]:
core7_reference_summary = reference_risk_df.copy()

core7_reference_summary["core7_implication"] = (
    "Historical instability motivates conservative governance activation"
)

core7_reference_summary

,antibody_id,risk_context,core7_implication
0,001,No strong signal detected,Historical instability motivates conservative ...
1,045,No strong signal detected,Historical instability motivates conservative ...
2,183,No strong signal detected,Historical instability motivates conservative ...
